Build a regression model.

Provide model output and an interpretation of the results. 

In [1]:
import requests
import pandas as pd
import statsmodels.api as sm

url = "http://api.citybik.es/v2/networks"
response = requests.get(url)

if response.status_code == 200:
    response_data = response.json()
    networks = response_data.get('networks', [])
    vancouver_network = None
    for network in networks:
        if network['location']['city'].lower() == 'vancouver':
            vancouver_network = network
            break

    if vancouver_network:
        network_id = vancouver_network['id']
        details_url = f"http://api.citybik.es/v2/networks/{network_id}"
        details_response = requests.get(details_url)
        
        if details_response.status_code == 200:
            vancouver_details = details_response.json()
            stations_in_vancouver = vancouver_details.get('network', {}).get('stations', [])
        else:
            print(f"Failed to retrieve details for Vancouver network: {details_response.status_code}")
    else:
        print("Vancouver network not found.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

foursquare_headers = {
    "accept": "application/json",
    "Authorization": "fsq3XRUAarAkYkZaUxo7U14qSYIS91CYwbt1bF94/FymeqU="
}
foursquare_url = "https://api.foursquare.com/v3/places/search"

station_data = []

for station in stations_in_vancouver:
    latitude = station['latitude']
    longitude = station['longitude']
    params = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'categories': '13065,13027'
    }
    response = requests.get(foursquare_url, headers=foursquare_headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        places = data.get('results', [])
        num_places = len(places)
        station_data.append({
            'Latitude': latitude,
            'Longitude': longitude,
            'Number of Restaurants and Bars': num_places
        })
    else:
        print(f"Failed to retrieve data from Foursquare API: {response.status_code}")

df = pd.DataFrame(station_data)

df['Number of Bike Stations'] = [1] * len(df)

X = df[['Number of Restaurants and Bars']]
X = sm.add_constant(X)
y = df['Number of Bike Stations']

ols_model = sm.OLS(y, X).fit()

print(ols_model.summary())

df['Predicted Number of Bike Stations'] = ols_model.predict(X)

most_suitable_location = df.loc[df['Predicted Number of Bike Stations'].idxmax()]

print("Most Suitable Location:")
print(most_suitable_location)

                               OLS Regression Results                              
Dep. Variable:     Number of Bike Stations   R-squared:                        -inf
Model:                                 OLS   Adj. R-squared:                   -inf
Method:                      Least Squares   F-statistic:                    -252.0
Date:                     Tue, 13 Aug 2024   Prob (F-statistic):               1.00
Time:                             18:47:49   Log-Likelihood:                 8187.3
No. Observations:                      254   AIC:                        -1.637e+04
Df Residuals:                          252   BIC:                        -1.636e+04
Df Model:                                1                                         
Covariance Type:                 nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

C:\Users\kasun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


# Stretch

How can you turn the regression model into a classification model?

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import Binarizer

# Assuming df1 and df2 are already created and combined_df is prepared

# Prepare features and target variable
combined_df = combined_df.dropna(subset=['Latitude', 'Longitude'])
X = combined_df[['Longitude']]
y = combined_df['Latitude']

bins = [-float('inf'), 49.25, 49.30, float('inf')]
labels = ['Low', 'Medium', 'High']
y_binned = pd.cut(y, bins=bins, labels=labels)

scaler = Binarizer()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binned, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))